# PostgreSQL (PSQL) Database 2
We store "data models" as tables. A table can be thought of a single CSV file. An inventor will be a table whereas a patent will be another table. We will then "join" them at runtime if necessary.

For the actual database to be used, we will use PostgreSQL to allow for complex queries and support large scale operations (in preparation when it's used extensively).


Read more about PostgreSQL vs MySQL here:

https://www.fivetran.com/blog/postgresql-vs-mysql

**Most of the things we learn here applies to all SQL databases**

## Updating data

### UPDATE
```sql
UPDATE inventors SET last_name='updated last name' where key_id='1';
```

- `UPDATE` indicates that the column values in the table named `inventors`
- `SET` indicates the columns to update and the new values to use
- `WHERE` defines the filters to find the rows to update (not setting this will update all rows in the table)

More details can be found here:

https://www.postgresqltutorial.com/postgresql-update/

https://www.postgresql.org/docs/14/sql-update.html

#### upserting data
```sql
INSERT INTO inventors(
    key_id, first_name, last_name, location_city, location_state, location_longitude, location_latitude
)
VALUES (
    '2', 'test first name', 'test last name', 'test city', 'test state', 0.0, 0.0
)
ON CONFLICT(key_id)
DO
    UPDATE SET first_name='upserted first name';
```
- `INSERT INTO ... VALUES` indicates that we insert a new row in the table named `inventors`
- `ON CONFLICT` indicates that we want to handle duplicates on the column `key_id`
- `DO` defines how we we handle the duplicates
    - `UPDATE SET` defines the columns and values that we update

This is the process of inserting or updating the data (if it already exists) in 1 query by using the `PRIMARY KEY` to identify duplicates.

More details can be found here:

https://www.postgresqltutorial.com/postgresql-upsert/

### ALTER TABLE

This is the process of updating how the table is configured. Necessary when a new column is to be added or existing columns are to be updated (or deleted).

```sql
ALTER TABLE inventors
ADD id VARCHAR;
```
- `ALTER TABLE` indicates that the configuration of the table named `inventors` will be updated
- `ADD` indicates that a new column named `id` will be added with `VARCHAR` as its datatype

More details can be found here:

https://www.postgresqltutorial.com/postgresql-alter-table/

https://www.postgresql.org/docs/14/sql-altertable.html

## Modelling relationships

This is the process of identifying each data model that we need (`inventors` and `patents` for example) and how they are related to each other.

### Normalized vs Denormalized Data Model

`Normalized` data model means that we minimize duplicates as much as possible. For example, for each `patents`, there can be multiple `inventors`. Instead of storing all of the `inventor` data to each `patents`, we only store the `ids` and store the rest in a separate table named `inventors`. This minimizes the storage costs and allowing efficient fetching of `inventors` (or `patents`) if the other table is not necessary. However, an overhead is incurred if they need to be combined on runtime.

`Denormalized` data model means that we store as much data as possible. For example, we store all `inventors` and their details alongside the list of `patents`. This minimizes the overhead incurred if they need to be combined (since they are already combined to begin with). However, this will have a higher storage costs since the data is duplicated in multiple rows.

The usage of either one depends on the actual needs of the application and the budget. Generally, a normalized data model is preferred because the performance overhead is often not high enough to justify the storage costs (unless it contains big data).

One tool that we can use is **draw.io** to visually model the data and their relationships

Here's a sample model using **draw.io**:

https://drive.google.com/file/d/1qhau5i8qR-h0NtfbnAG-CgGdb8Ar9NW8/view?usp=sharing

### JOIN

This is the process of combining different tables together in a single query. Useful for normalized data models so that all details can still be fetched in a single query.